In [12]:
import urllib.request
import xml.etree.ElementTree
from PIL import Image
import time
import pandas as pd
from IPython.display import display
import requests
import re
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

Using TensorFlow backend.


In [18]:
book_db.describe()

,Title,Author,Goodreads Rating,Description
count,1426,1347,1426,1426
unique,1336,731,171,1229
top,The+Da+Vinci+Code,-,4.00,Tinkle Digest contains stories from back issue...
freq,6,60,35,7


In [17]:
book_db = pd.read_csv("Quills_and_goodreads_db.csv")

In [18]:
book_db.describe()

,Unnamed: 0,Goodreads Rating
count,1426.000000,1426.000000
mean,1045.767882,3.829123
std,604.045862,0.632145
min,0.000000,0.000000
25%,503.250000,3.700000
50%,1084.500000,3.930000
75%,1565.750000,4.110000
max,2009.000000,5.000000


In [19]:
raw_text = ''
for desc in book_db['Description']:
    raw_text = raw_text + desc

In [20]:
len(raw_text)

1092541

In [21]:
raw_text = raw_text.lower()

In [22]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [23]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  1092541
Total Vocab:  248


In [24]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  1092441


In [25]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [26]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [27]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [28]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1092441/1092441 [==============================] - 1226s 1ms/step - loss: 2.9091

Epoch 00001: loss improved from inf to 2.90906, saving model to weights-improvement-01-2.9091.hdf5
Epoch 2/20
1092441/1092441 [==============================] - 1199s 1ms/step - loss: 2.7728

Epoch 00002: loss improved from 2.90906 to 2.77280, saving model to weights-improvement-02-2.7728.hdf5
Epoch 3/20
1092441/1092441 [==============================] - 1319s 1ms/step - loss: 2.7040

Epoch 00003: loss improved from 2.77280 to 2.70398, saving model to weights-improvement-03-2.7040.hdf5
Epoch 4/20
1092441/1092441 [==============================] - 1763s 2ms/step - loss: 2.6491

Epoch 00004: loss improved from 2.70398 to 2.64911, saving model to weights-improvement-04-2.6491.hdf5
Epoch 5/20
1092441/1092441 [==============================] - 1454s 1ms/step - loss: 2.6054

Epoch 00005: loss improved from 2.64911 to 2.60537, saving model to weights-improvement-05-2.6054.hdf5
Epoch 6/20
1092441/10924

KeyboardInterrupt: 

In [29]:
# load the network weights
filename = "weights-improvement-02-2.7728.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [31]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" tiff's attempt to find individuality. the brilliance of the novel is orwell's prescience of modern l "
o the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to the sore to

In [34]:
def generate_text(filename):
    model.load_weights(filename)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    int_to_char = dict((i, c) for i, c in enumerate(chars))
    start = numpy.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    print( "Seed:")
    print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    # generate characters
    for i in range(1000):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    print ("\nDone.")

In [35]:
generate_text("weights-improvement-03-2.7040.hdf5")
generate_text("weights-improvement-05-2.6054.hdf5")
generate_text("weights-improvement-09-2.4812.hdf5")



Seed:
"  they want—no, need—is each other? playing with fate is a standalone gay romance novel with no cheat "
 the sorld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee torld to tee 

In [36]:
generate_text("weights-improvement-11-2.4385.hdf5")
generate_text("weights-improvement-13-2.4033.hdf5")
generate_text("weights-improvement-15-2.3732.hdf5")


Seed:
" the earth, what is interesting hannah these days isn't her worst student㟩t's his father,what she can "
e oo the sore tfet tee soeee of the cart oo the sore of the cart oo the sore of the cart oo the sore of the coesint of the coesint of the coesint of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the coasiit of the coesint of the c

In [37]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
   5632/1092441 [..............................] - ETA: 30:40 - loss: 2.3537

KeyboardInterrupt: 